continuing working on optimizing the model

In [98]:
import pandas as pd
import numpy as np
from collections import defaultdict
from datetime import datetime

import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [20]:
df_tune = pd.read_csv('data/2019aug/cleaned_dlistings.csv', low_memory=False)

df_orig = pd.read_csv('data/2019aug/dlistings.csv', low_memory=False)

In [4]:
df_tune_1 = df_tune.copy()

In [166]:
prices.to_csv('data/2019aug/price_final.csv')

In [167]:
df_tune.drop(df_tune[df_tune['neighbourhood'].isna()].index, axis=0, inplace=True)

In [170]:
df_tune.drop(['neighbor_group : Brooklyn',
            'neighbor_group : Manhattan', 'neighbor_group : Queens', 
            'neighbor_group : Staten Island'], axis=1, inplace=True)

In [172]:
final_dummies = pd.get_dummies(df_tune['neighbourhood'], drop_first=True)

In [173]:
df_tune = pd.concat([df_tune, final_dummies], axis=1)

In [175]:
df_tune.to_csv('data/2019aug/final_dlistings.csv')

In [156]:
for i in df_tune.columns:
    print(i)

host_since
host_response_rate
host_is_superhost
host_listings_count
host_total_listings_count
host_has_profile_pic
host_identity_verified
accommodates
bathrooms
bedrooms
beds
show_sqft
offer_weekly
offer_monthly
security_deposit
cleaning_fee
guests_included
extra_people
minimum_nights
maximum_nights
minimum_minimum_nights
maximum_minimum_nights
minimum_maximum_nights
maximum_maximum_nights
minimum_nights_avg_ntm
maximum_nights_avg_ntm
number_of_reviews
number_of_reviews_ltm
review_scores_rating
review_scores_accuracy
review_scores_cleanliness
review_scores_checkin
review_scores_communication
review_scores_location
review_scores_value
instant_bookable
require_guest_profile_picture
require_guest_phone_verification
calculated_host_listings_count
reviews_per_month
summary_p
summary_s
space_p
space_s
description_p
description_s
neighborhood_overview_p
neighborhood_overview_s
notes_p
notes_s
transit_p
transit_s
access_p
access_s
interaction_p
interaction_s
house_rules_p
house_rules_s
host_ab

In [15]:
aggressive_drop = ['host_response_rate', 'host_listings_count', 
            'host_total_listings_count', 'host_has_profile_pic',
            'show_sqft', 'minimum_nights', 'minimum_minimum_nights',
            'maximum_minimum_nights', 'minimum_maximum_nights' ,
            'maximum_maximum_nights', 'minimum_nights_avg_ntm',
            'maximum_nights_avg_ntm', 'number_of_reviews', 
            'number_of_reviews_ltm', 'neighbor_group : Brooklyn',
            'neighbor_group : Manhattan', 'neighbor_group : Queens', 
            'neighbor_group : Staten Island', 'verification_google', 
            'verification_offline_government_id', 'verification_sent_id', 
            'verification_facebook', 'verification_manual_online',
            'verification_selfie', 'verification_email', 
            'verification_reviews', 'verification_government_id',
            'verification_manual_offline', 'verification_phone', 
            'verification_jumio', 'verification_sesame', 'verification_weibo',
            'verification_work_email', 'verification_identity_manual',
            'verification_sesame_offline', 'verification_zhima_selfie', 
            'verification_kba'
    
]


In [14]:
sum(df_tune['require_guest_profile_picture'].values)

1090

In [19]:
df_tune_1.drop(aggressive_drop, axis=1, inplace=True)

In [30]:
df_orig.drop(df_orig[df_orig['host_since'].isna()].index, axis=0, inplace=True)

In [36]:
df_orig.drop(df_orig[df_orig['cancellation_policy'].isna()].index, axis=0, inplace=True)

In [38]:
neighbourhood = df_orig['neighbourhood_cleansed']

In [42]:
df_tune['neighbourhood'] = neighbourhood

df_tune_1['neighbourhood'] = neighbourhood

In [56]:
df_tune_1.drop(df_tune_1[df_tune_1['neighbourhood'].isna()].index, axis=0, inplace=True)

In [59]:
prices = pd.read_csv('data/2019aug/prices.csv')

In [73]:
prices.drop(df_tune[df_tune['neighbourhood'].isna()].index, axis=0, inplace=True)

In [77]:
prices.drop('Unnamed: 0', axis=1, inplace=True)

In [82]:
tune_1_dummies = pd.get_dummies(df_tune_1['neighbourhood'], drop_first=True)

In [87]:
df_tune_1 = pd.concat([df_tune_1, tune_1_dummies], axis=1)

In [91]:
df_tune_1.drop('neighbourhood', axis=1, inplace=True)

In [92]:
X1 = sm.add_constant(df_tune_1)

In [95]:
ols_tune1 = sm.OLS(prices.price, X1.astype(float))

ols_tune1 = ols_tune1.fit()

In [181]:
feature_params_ols = ols_tune1.params

In [99]:
df_tune_2 = df_tune_1.copy()

In [101]:
df_tune_2.drop('maximum_nights', axis=1, inplace=True)

In [102]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

In [107]:
X2_train, X2_test, y2_train, y2_test = train_test_split(df_tune_2, prices.price, test_size=0.33)

In [108]:
X2_train.shape

(32713, 445)

In [109]:
X2_test.shape

(16113, 445)

In [110]:
scaler = StandardScaler()
scaler.fit(X2_train)

StandardScaler()

In [113]:
logmodel  = LogisticRegression(penalty='l1', max_iter=1000, C=1, solver='liblinear')

In [118]:
select_features = SelectFromModel(logmodel)

In [121]:
select_features.fit(scaler.transform(X2_train), y2_train)

SelectFromModel(estimator=LogisticRegression(C=1, max_iter=1000, penalty='l1',
                                             solver='liblinear'))

In [123]:
selected = X2_train.columns[(select_features.get_support())]

In [137]:
removed = []
for i in X2_train.columns:
    if i not in selected:
        removed.append(i)

In [138]:
# identify removed features
removed

['property_type : Barn',
 'property_type : Bus',
 'property_type : Dome house',
 'Howland Hook',
 'New Dorp',
 'Willowbrook']

In [140]:
df_tune_3 = df_tune_2.drop(removed, axis=1)

In [153]:
df_tune_3.to_csv('data/2019aug/dlistings_rf')

In [142]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [151]:
X3_train, X3_test, y3_train, y3_test = train_test_split(df_tune_3, prices.price, test_size=0.33)

random_grid = {'n_estimators': [100, 200, 400, 600, 800, 1000, 1500, 2000, 3000, 5000],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [10, 20, 30, 40, 60, 80, 100, 150, None],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

rf = RandomForestRegressor()
rf_rs = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                           n_iter = 100, cv = 5, verbose=2, random_state = 1, n_jobs = -1)

In [147]:
X3_train.shape

(32713, 439)

In [148]:
X3_test.shape

(16113, 439)

In [152]:
rf_rs.fit(X3_train, y3_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 27.0min


KeyboardInterrupt: 